In [34]:
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

import os
import time
import matplotlib

# import skimage
from skimage import io, filters, exposure, measure, transform
from scipy.signal import find_peaks

pd.set_option('mode.chained_assignment',None)

%matplotlib widget 
# %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})


SMALLER_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [10]:
# testDataFolder = 'D://PMMH_ComputerData//Matlab Analysis//Data_Joseph//Raw//20.06.15_Deptho//depthoAnalysis_test'
testDataFolder = 'C://Users//JosephVermeil//Desktop//test_AnalyseDepthos'
listDepthos = [f for f in os.listdir(testDataFolder) if (os.path.isfile(os.path.join(testDataFolder, f)) and f.endswith(".tif"))]
listDepthos

['multiDeptho5.tif']

In [11]:
fileName = listDepthos[0]
filePath = os.path.join(testDataFolder,fileName)
I = skimage.io.imread(filePath)

In [12]:
if not I.ndim == 3:
    print('Unexpected image format !')
else:
    I0 = I[0]

In [13]:
fig, ax = plt.subplots(1,1)
ax.imshow(I[2], cmap = 'gray')
fig.tight_layout()
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
I.shape
smallI = I[198:203]
smallROI = I[:,100:400,100:500]
# plt.imshow(smallROI, cmap = 'gray')
threshold = skimage.filters.threshold_otsu(smallROI)
threshold

9135

In [16]:
fig, ax = plt.subplots(1,1)
ax.hist(smallROI.ravel(), bins=256, histtype='step', color='black')
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# np.zeros([nz, ny, nx])

In [17]:
nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
maxVal = np.zeros(nz)
for z in range(nz):
    maxVal[z] = np.max(I[z])

#
Z = np.array([i for i in range(nz)])
fig, ax = plt.subplots(1,1)
ax.plot(Z, maxVal)
fig.show()
#

zMax = np.argmax(maxVal)
IMax = I[zMax]

#
fig, ax = plt.subplots(1,1)
ax.imshow(IMax, cmap = 'gray')
fig.show()
#

threshold = skimage.filters.threshold_li(IMax)
binIMax = IMax > threshold

#
fig, ax = plt.subplots(1,1)
ax.imshow(binIMax, cmap = 'gray')
fig.show()
#

threshold = skimage.filters.threshold_otsu(IMax)
binIMax = IMax > threshold

#
fig, ax = plt.subplots(1,1)
ax.imshow(binIMax, cmap = 'gray')
fig.show()
#

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
class MultiBeadsDeptho:
   
    def __init__(self, I, scale):
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.__I = I
        self.__scale = scale
        self.__I_labeled = np.zeros([nz, ny, nx])
        self.__nz = nz
        self.__ny = ny
        self.__nx = nx
        self.__beadList = []
        self.__nB = 0
        
        
        
    def get_beadList(self):
        return(self.__beadList)
    
    def build_beadList(self):
        maxVal = np.zeros(self.__nz)
        for z in range(self.__nz):
            maxVal[z] = np.max(self.__I[z])
        z_max = np.argmax(maxVal)
        I_max = self.__I[z_max]
        threshold = skimage.filters.threshold_otsu(I_max)
        I_max_bin = (I_max > threshold)
        I_labeled, nB = measure.label(I_max_bin, return_num = True)
        self.__I_labeled, self.__nB = I_labeled, nB
        
#         for k in range(1,self.__nB+1):
#             ym0, xm0 = ndi.center_of_mass(I_max, I_labeled, k)
#             ym0, xm0 = int(np.floor(ym0)), int(np.floor(xm0))
#             I_labeled[ym0-1:ym0+2, xm0-1:xm0+2] = 2*nB
            
#         fig, ax = plt.subplots(1,1)
#         ax.imshow(I_labeled)
            
        for k in range(1,self.__nB+1):
#             print(k)
#             print(ndi.center_of_mass(I_max, I_labeled, k))
            ym0, xm0 = ndi.center_of_mass(I_max, I_labeled, k)
            self.__beadList.append(BeadDeptho(I, I_labeled, k, xm0, ym0, self.__scale))
            self.__beadList[-1].build_ROI()
            
    
    
    @property
    def I(self):
        return(self.__I)
    
    @I.setter
    def I(self, I):
#         self.__I  =  I
        print('FORBIDEN: You cannot change the image after this object instantiation.')
    
    @property
    def nz(self):
        return(self.__nz)
    
    @property
    def ny(self):
        return(self.__ny)
    
    @property
    def nx(self):
        return(self.__nx)
    
    @property
    def nB(self):
        return(self.__nB)

    

In [236]:
class BeadDeptho:
    def __init__(self, I, I_labeled, my_label, xm0, ym0, scale):
        
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.I = I
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.scale = scale
        self.xm0 = xm0
        self.ym0 = ym0
        self.I_labeled = I_labeled
        self.label = my_label
        self.D = 0
        self.closeToTheEdge = False
    
    
    def build_ROI(self):
        roughSize = np.floor(5*self.scale)
        roughSize += roughSize%2
        x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
        x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
        y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
        y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)

        if min([x1_roughROI,self.nx-x2_roughROI,y1_roughROI,self.ny-y2_roughROI]) < 0:
            self.closeToTheEdge = True
            
        if not self.closeToTheEdge:
            I_roughRoi = I[:, y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]
            maxVal = np.zeros(self.nz)
            for z in range(self.nz):
                maxVal[z] = np.max(I_roughRoi[z])
            z_max = np.argmax(maxVal)

#             fig, ax = plt.subplots(1,2, figsize = (8,4))
#             ax[0].imshow(I_roughRoi[z_max], cmap = 'gray')
#             ax[1].hist(I_roughRoi[z_max].ravel(), bins=256, histtype='step', color='black')

            counts, binEdges = np.histogram(I_roughRoi[z_max].ravel(), bins=256)

            peaks, peaksProp = find_peaks(counts, height=None, threshold=None, distance=1, prominence=None, \
                               width=1, wlen=None, rel_height=0.5, plateau_size=None)
    #         print(max(counts), counts[peaks[0]])
            if counts[peaks[0]] > 1200:
                self.D = 4.5
                roughSize = np.floor(12*self.scale)
                roughSize += roughSize%2
                x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
                x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
                y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
                y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)
                I_roughRoi = I[:, y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]
            else:
                self.D = 2.7
                roughSize = np.floor(8*self.scale)
                roughSize += roughSize%2
                x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
                x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
                y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
                y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)
                I_roughRoi = I[:, y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]
            
            if min([x1_roughROI,self.nx-x2_roughROI,y1_roughROI,self.ny-y2_roughROI]) < 0:
                self.closeToTheEdge = True
    #         h_roughROI, w_roughROI = x2_roughROI-x1_roughROI, y2_roughROI-y1_roughROI
    #         print(h_roughROI//2, w_roughROI//2)
            
            if not self.closeToTheEdge:

                threshold = 0.8*skimage.filters.threshold_otsu(I_roughRoi)
                I_roughRoi_binary = (I_roughRoi > threshold)
                I_roughRoi_binary_fh = ndi.binary_fill_holes(I_roughRoi_binary).astype(int)

                area, circ = np.zeros(self.nz), np.zeros(self.nz)
                for i in range(self.nz):
                    props = measure.regionprops_table(I_roughRoi_binary_fh[i], properties=('perimeter', 'area'))
                    if not np.sum(I_roughRoi_binary_fh[i]) == 0 and props['perimeter'][0] != 0:
                        area[i] = props['area'][0]
                        circ[i] = (4 * np.pi * props['area']) / (props['perimeter'] * props['perimeter'])[0]       
                validSlice = (area > 50) & (circ > 0.8)
                zFirst = np.argmax(validSlice)
                zLast = self.nz - np.argmax(validSlice[::-1])

                XYm = np.zeros((2, self.nz), dtype = int)
                cleanHalfSize = int(round(1*self.D*self.scale))
                I_cleanROI = np.zeros([self.nz, 2*cleanHalfSize+1, 2*cleanHalfSize+1])

                print(zFirst, zLast)
                for i in range(zFirst, zLast):
                    try:
                        Ym, Xm = ndi.center_of_mass(I_roughRoi[i], labels=I_roughRoi_binary_fh[i], index=1)
                        XYm[1, i], XYm[0, i] = int(round(Ym)), int(round(Xm))
                        roughCenter = int((roughSize+1)//2)
                        tform = transform.EuclideanTransform(rotation=0, translation = (Xm-roughCenter, Ym-roughCenter))
                        I_tmp = transform.warp(I_roughRoi[i], tform, order = 2, preserve_range = True)
        #                 centerPoint = I_tmp.shape[0]//2
                        I_cleanROI[i] = np.copy(I_tmp[roughCenter-cleanHalfSize:roughCenter+cleanHalfSize+1,roughCenter-cleanHalfSize:roughCenter+cleanHalfSize+1])

                        # VISUALISE
                        if i == zFirst:
                            print(i)
                            print('Job done for the bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))
                            fig, ax = plt.subplots(1,3, figsize = (12,4))
                            ax[0].imshow(I_roughRoi[i], cmap = 'gray')
                            mid = I_roughRoi[i].shape[0]//2
                            ax[0].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
                            ax[0].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
                            I_max_binary = (I_roughRoi[i] > threshold)
                            y, x = ndi.center_of_mass(I_roughRoi[i], labels=I_max_binary, index=1)
                            ax[0].plot([x],[y], 'b+')

                            ax[1].imshow(I_tmp, cmap = 'gray')
                            mid = I_tmp.shape[0]//2
                            ax[1].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
                            ax[1].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
                            I_tmp_binary = (I_tmp > threshold)
                            y, x = ndi.center_of_mass(I_tmp, labels=I_tmp_binary, index=1)
                            ax[1].plot([x],[y], 'b+')

                            ax[2].imshow(I_cleanROI[i], cmap = 'gray')
                            mid = I_cleanROI[i].shape[0]//2
                            ax[2].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
                            ax[2].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
                            I_cleanRoi_binary = (I_cleanROI > threshold)
                            y, x = ndi.center_of_mass(I_cleanROI[i], labels=I_cleanRoi_binary[i], index=1)
                            ax[2].plot([x],[y], 'b+')


                    except:
                        print('bug for the bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))

                print(I_tmp.shape[0],I_tmp.shape[1])

    #         ax.hist(I_roughRoi[z_max].ravel(), bins=256, histtype='step', color='black')
    #         print(XYm)
                          
    
    def build_zProfiles(self):
        return(0)

In [237]:
mbd = MultiBeadsDeptho(smallI, 15.8)
mbd.nB

0

In [238]:
mbd.build_beadList()
mbd.nB
for bD in mbd.get_beadList():
    print(bD.D)


119 215
119
Job done for the bead in x0, y0 = 1330.23, 160.01


C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Bi-quadratic interpolation behavior has changed due to a bug in the implementation of scikit-image. The new version now serves as a wrapper around SciPy's interpolation functions, which itself is not verified to be a correct implementation. Until skimage's implementation is fixed, we recommend to use bi-linear or bi-cubic interpolation instead.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

127 127
0 401
0
Job done for the bead in x0, y0 = 1544.49, 204.91


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for the bead in x0, y0 = 1057.46, 493.83
bug for th

142 301
142
Job done for the bead in x0, y0 = 443.36, 522.10


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

127 127
0
2.7
2.7
4.5
2.7
4.5
4.5
4.5
2.7
2.7
2.7
4.5
4.5
2.7
4.5
4.5
4.5
2.7


In [ ]:
mbd.I

In [154]:
A = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
A

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [155]:
O = np.zeros([3, 2, 2])
O

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [156]:
O[1]

array([[0., 0.],
       [0., 0.]])

In [157]:
O[1,:,:] = A[:2,:2]
O

array([[[0., 0.],
        [0., 0.]],

       [[1., 2.],
        [4., 5.]],

       [[0., 0.],
        [0., 0.]]])

In [160]:
A[0,0] = 0
A

array([[0, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [159]:
O

array([[[0., 0.],
        [0., 0.]],

       [[1., 2.],
        [4., 5.]],

       [[0., 0.],
        [0., 0.]]])